In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model


In [2]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-3, 3))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
         data = yf.download(symbol, period="60d",
                            interval="5m", progress=False)
         if data.empty:
              pass
         else:
               if data.shape[0] > 12:
                   unattached_dfs.append(
                       process(data))

    data = np.array(unattached_dfs[0])
    for i in unattached_dfs[1:]:
        data = np.append(data, np.array(i), axis=0)
        
    symbol = []

    data = pd.DataFrame(data, columns=clmns)
    sugg = data["suggestion"]
    data.drop("suggestion",axis=1,inplace=True)
    sugg = pd.get_dummies(sugg)
    data = pd.concat([data,sugg],axis=1)
    data = data.astype(float)

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,10000)}.csv")
    #data = []
    return xTrain, xTest, yTrain, yTest,data



#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

pieces = 100
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays:
    get_data(symbols)
#symbols = ["btc-usd","eth-usd","trx-usd","bnb-usd","xrp-usd"]
#xTrain, xTest, yTrain, yTest,data= get_data(symbols)
#data 

In [3]:
data = pd.concat([pd.read_parquet("/content/drive/MyDrive/cmm1.parquet"),pd.read_parquet("/content/drive/MyDrive/cmm2.parquet"),pd.read_parquet("/content/drive/MyDrive/cmm3.parquet")])
data

,Unnamed: 0,Open 12-11,Open 11-10,Open 10-9,Open 9-8,Open 8-7,Open 7-6,Open 6-5,Open 5-4,Open 4-3,...,Volume 9-8,Volume 8-7,Volume 7-6,Volume 6-5,Volume 5-4,Volume 4-3,Volume 3-2,Volume 2-1,no,yes
0,0,2.428931,1.263724,-1.167071,-0.389365,-2.497483,0.147917,-3.000000,-0.414717,1.144235,...,-0.549051,0.183985,-0.182533,-0.182533,-0.547599,-3.000000,3.000000,-0.182533,1.0,0.0
1,1,-0.256920,2.428931,1.263724,-1.167071,-0.389365,-2.497483,0.147917,-3.000000,-0.414717,...,-0.182533,-0.549051,0.183985,-0.182533,-0.182533,-0.547599,-3.000000,3.000000,1.0,0.0
2,2,-2.646832,0.031624,3.000000,1.712226,-0.974265,-0.114753,-2.444624,0.479046,-3.000000,...,1.263754,1.228404,0.678340,1.778468,1.228404,1.228404,0.680519,-3.000000,0.0,1.0
3,3,-0.812076,-2.646832,0.031624,3.000000,1.712226,-0.974265,-0.114753,-2.444624,0.479046,...,3.000000,0.088906,0.029635,-0.892635,0.951906,0.029635,0.029635,-0.888980,1.0,0.0
4,4,0.533693,-0.812076,-2.646832,0.031624,3.000000,1.712226,-0.974265,-0.114753,-2.444624,...,-3.000000,3.000000,0.088906,0.029635,-0.892635,0.951906,0.029635,0.029635,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255977,255977,-0.785809,0.918103,-2.370358,-0.243700,-0.442639,0.768899,3.000000,-1.620358,0.335212,...,0.427483,-1.050143,0.892098,-0.079023,-0.349413,0.033323,-3.000000,3.000000,1.0,0.0
255978,255978,-0.133914,-1.229672,0.674014,-3.000000,-0.624004,-0.846268,0.507316,3.000000,-2.162067,...,1.511742,2.283757,0.005871,3.000000,1.502935,1.086106,1.676125,-3.000000,1.0,0.0
255979,255979,0.369513,-0.133914,-1.229672,0.674014,-3.000000,-0.624004,-0.846268,0.507316,3.000000,...,-1.505882,0.017647,1.564706,-3.000000,3.000000,0.000000,-0.835294,0.347059,1.0,0.0
255980,255980,-2.387664,0.369513,-0.133914,-1.229672,0.674014,-3.000000,-0.624004,-0.846268,0.507316,...,0.511765,-1.505882,0.017647,1.564706,-3.000000,3.000000,0.000000,-0.835294,1.0,0.0


In [4]:
data.drop("Unnamed: 0",axis=1,inplace=True)
xTrain, xTest, yTrain, yTest = spliting(data)


(2960148, 66) (2960148, 2)
(328906, 66) (328906, 2)


In [5]:
data = []

In [6]:
model = Sequential()



model.add(Dense(2000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))




model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              134000    
                                                                 
 dense_1 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_2 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_3 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_4 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_5 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_6 (Dense)             (None, 2000)              4

In [ ]:
model.fit(xTrain,yTrain,epochs=30,batch_size=8192,validation_data=(xTest,yTest))

Epoch 1/30
47/47 [==============================] - 143s 3s/step - loss: 0.6925 - accuracy: 0.5165 - val_loss: 0.6920 - val_accuracy: 0.5206
Epoch 2/30
47/47 [==============================] - 142s 3s/step - loss: 0.6914 - accuracy: 0.5257 - val_loss: 0.6915 - val_accuracy: 0.5231
Epoch 3/30
47/47 [==============================] - 141s 3s/step - loss: 0.6907 - accuracy: 0.5280 - val_loss: 0.6909 - val_accuracy: 0.5281
Epoch 4/30
47/47 [==============================] - 142s 3s/step - loss: 0.6889 - accuracy: 0.5366 - val_loss: 0.6903 - val_accuracy: 0.5325
Epoch 5/30
47/47 [==============================] - 142s 3s/step - loss: 0.6871 - accuracy: 0.5449 - val_loss: 0.6902 - val_accuracy: 0.5334
Epoch 6/30
47/47 [==============================] - 141s 3s/step - loss: 0.6840 - accuracy: 0.5542 - val_loss: 0.6907 - val_accuracy: 0.5322
Epoch 7/30
47/47 [==============================] - 141s 3s/step - loss: 0.6810 - accuracy: 0.5627 - val_loss: 0.6902 - val_accuracy: 0.5410
Epoch 8/30
47

KeyboardInterrupt: ignored

In [14]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model = load_model("/content/drive/MyDrive/[0.23090815544128418, 0.9449216723442078].h5")

In [ ]:
model.fit(xTrain,yTrain,epochs=10,batch_size=55555,validation_data=(xTest,yTest))